In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
import datetime

df = pd.read_csv("Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.loc[df.Date > datetime.datetime(year=2010, month=1, day=1)]
df.head()

,PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,...,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
1,13605159706372,136051597,6372,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Thursday,2013-03-14,09:21,SOUTHERN,NONE,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,32.0
2,13618826906244,136188269,6244,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Tuesday,2013-10-08,19:30,MISSION,NONE,...,NaN,3.0,NaN,NaN,2.0,NaN,NaN,3.0,NaN,NaN
5,13085582009320,130855820,9320,FRAUD,"CREDIT CARD, THEFT BY USE OF",Tuesday,2013-10-08,21:11,PARK,NONE,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,13.0
6,11091687865016,110916878,65016,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Sunday,2011-11-13,10:15,BAYVIEW,"ARREST, CITED",...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,86.0
11,13607041271000,136070412,71000,NON-CRIMINAL,LOST PROPERTY,Saturday,2013-04-20,16:00,NORTHERN,NONE,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,22.0


In [2]:
df['Time'] = pd.to_datetime(df['Time'])
df["Hours"] = df.Time.dt.strftime('%H')
df_hours = df.groupby(["Category", "Hours"]).agg('count')

In [3]:
df_hours = df_hours.reset_index()[["Category", "Hours", "IncidntNum"]]
df_hours.head()

,Category,Hours,IncidntNum
0,ARSON,00,147
1,ARSON,01,135
2,ARSON,02,139
3,ARSON,03,144
4,ARSON,04,127


In [36]:
focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])
crimes = df_hours.Category.unique()

for crime in df_hours.Category.unique():
    df_hours[crime] = df_hours.loc[df_hours.Category==crime].IncidntNum
    
result_df = df_hours.groupby("Hours").agg("sum")
result_df.drop('IncidntNum', inplace=True, axis=1)
result_df.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,"SEX OFFENSES, NON FORCIBLE",STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Hours,,,,,,,,,,,,,,,,,,,,,
00,147.0,5920.0,47.0,29.0,1935.0,244.0,405.0,1640.0,402.0,265.0,...,41.0,326.0,27.0,3029.0,0.0,312.0,3544.0,1802.0,2053.0,662.0
01,135.0,5053.0,1.0,17.0,1338.0,181.0,377.0,978.0,385.0,5.0,...,2.0,246.0,18.0,1450.0,0.0,225.0,2486.0,1213.0,1392.0,479.0
02,139.0,4588.0,1.0,20.0,1517.0,154.0,333.0,793.0,352.0,2.0,...,1.0,216.0,18.0,1285.0,2.0,281.0,2314.0,915.0,1136.0,410.0
03,144.0,2458.0,1.0,11.0,1603.0,86.0,159.0,586.0,134.0,7.0,...,1.0,172.0,11.0,876.0,0.0,234.0,1686.0,600.0,905.0,287.0
04,127.0,1530.0,1.0,9.0,1431.0,72.0,58.0,435.0,71.0,8.0,...,1.0,146.0,8.0,604.0,0.0,178.0,1154.0,504.0,706.0,197.0


In [37]:
for name, data in result_df.iteritems():
    result_df[name] = result_df[name]/sum(data)
    
result_df

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,"SEX OFFENSES, NON FORCIBLE",STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Hours,,,,,,,,,,,,,,,,,,,,,
00,0.068724,0.056318,0.150160,0.056974,0.040427,0.051293,0.120357,0.034687,0.080240,0.190374,...,0.175214,0.044481,0.042320,0.063601,0.000000,0.028395,0.054419,0.035965,0.039855,0.054620
01,0.063114,0.048070,0.003195,0.033399,0.027954,0.038049,0.112036,0.020685,0.076846,0.003592,...,0.008547,0.033565,0.028213,0.030446,0.000000,0.020477,0.038173,0.024210,0.027023,0.039521
02,0.064984,0.043646,0.003195,0.039293,0.031694,0.032373,0.098960,0.016772,0.070259,0.001437,...,0.004274,0.029472,0.028213,0.026982,0.142857,0.025573,0.035532,0.018262,0.022053,0.033828
03,0.067321,0.023383,0.003195,0.021611,0.033491,0.018079,0.047251,0.012394,0.026747,0.005029,...,0.004274,0.023468,0.017241,0.018394,0.000000,0.021296,0.025889,0.011975,0.017569,0.023680
04,0.059374,0.014555,0.003195,0.017682,0.029897,0.015136,0.017236,0.009201,0.014172,0.005747,...,0.004274,0.019921,0.012539,0.012682,0.000000,0.016199,0.017720,0.010059,0.013706,0.016254
05,0.050491,0.012595,0.000000,0.013752,0.026241,0.059491,0.011293,0.005182,0.004990,0.005029,...,0.012821,0.016783,0.015674,0.010562,0.071429,0.038133,0.014664,0.010039,0.010774,0.008416
06,0.029921,0.015858,0.003195,0.011788,0.023358,0.119193,0.012481,0.010808,0.009182,0.002874,...,0.008547,0.017192,0.031348,0.015937,0.071429,0.074172,0.017075,0.015368,0.016365,0.011716
07,0.023843,0.022689,0.006390,0.015717,0.031945,0.101535,0.008618,0.025085,0.019960,0.010776,...,0.012821,0.024151,0.028213,0.025176,0.071429,0.073353,0.021590,0.022653,0.031138,0.021535
08,0.024310,0.033534,0.047923,0.029470,0.048199,0.075047,0.008024,0.033122,0.016966,0.068247,...,0.042735,0.028244,0.037618,0.040798,0.000000,0.064616,0.031386,0.033071,0.036399,0.024422


In [151]:
from bokeh.models import ColumnDataSource, FactorRange, Legend
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20
from bokeh.io import output_notebook, push_notebook, show

source = ColumnDataSource(result_df)
hours = [str(x) for x in range(0,23)]

p = figure(x_range = FactorRange(factors=result_df.index), plot_width=800, plot_height=500)

bar = {}
for indx,name in enumerate(focuscrimes):
    bar[name] = p.vbar(x='Hours',  top=name, source=source, 
                 legend_label=name, muted_color="white", muted_alpha=0, color=Category20[14][indx], alpha=0.8) 
    
p.legend.click_policy="mute" ### assigns the click policy (you can try to use ''hide'

items = []
for indx,name in enumerate(focuscrimes):
    items.append((name, [bar[name]]))
    
p.legend.items = []
legend = Legend(items=items, location=(0,0))
legend.click_policy="mute"
p.add_layout(legend, 'right')
show(p)